In [ ]:
#default_exp convert

In [ ]:
#export
from __future__ import annotations
from nbprocess.mdx import *
import os,sys

from nbconvert.exporters import Exporter
from fastcore.all import Path,parallel,call_parse,bool_arg,globtastic

# Convert Notebooks To Markdown

> Utilities that help you go from .ipynb -> .md

In [ ]:
#export
def nb2md(fname:str|Path, exp:Exporter):
    "Convert a notebook in `fname` to a markdown file."
    file = Path(fname)
    assert file.name.endswith('.ipynb'), f'{str(fname)} is not a notebook.'
    assert file.is_file(), f'file {str(fname)} not found.'
    print(f"converting: {str(file)}")
    try:
        o,r = exp.from_filename(fname)
        file.with_suffix('.md').write_text(o)
        return True
    except Exception as e:
        print(e)
        return False

We can use `nb2md` to convert a notebook to a markdown file with an `Exporter`.  Below, we use the exporter given to us by `nbdoc.mdx.get_mdx_exporter` and use that to create a markdown file from a notebook.

In [ ]:
_test_fname = Path('../tests/docs_test.ipynb')
_test_dest = Path('../tests/docs_test.md')
_test_dest.unlink(missing_ok=True)

nb2md(fname=_test_fname, exp = get_mdx_exporter()) # create the markdown file
assert _test_dest.exists() # make sure the markdown file does exist
assert len(_test_dest.readlines()) > 10

converting: ../tests/docs_test.ipynb


In [ ]:
#export
@call_parse
def parallel_nb2md(
    path:str='.', # path or filename
    recursive:bool=True, # search subfolders
    symlinks:bool=True, # follow symlinks?
    n_workers:int=None, # Number of parallel workers
    pause:int=0, # Pause between parallel launches
    force_all:bool=False, # Force rebuild docs that are up-to-date
    file_glob:str='*.ipynb', # Only include files matching glob
    file_re:str=None, # Only include files matching regex
    folder_re:str=None, # Only enter folders matching regex
    skip_file_glob:str=None, # Skip files matching glob
    skip_file_re:str=None, # Skip files matching regex
    skip_folder_re:str='^[_.]' # Skip folders matching regex
):
    if os.environ.get('IN_TEST',0): return
    exp=get_mdx_exporter()
    if not recursive: skip_folder_re='.'
    files = globtastic(path, symlinks=symlinks, file_glob=file_glob, file_re=file_re,
                       folder_re=folder_re, skip_file_glob=skip_file_glob,
                       skip_file_re=skip_file_re, skip_folder_re=skip_folder_re
                      ).map(Path)

    if len(files)==1: force_all,n_workers = True,0
    if not force_all:
        # only rebuild modified files
        files,_files = [],files.copy()
        for fname in _files:
            fname_out = fname.with_suffix('.md')
            if not fname_out.exists() or os.path.getmtime(fname) >= os.path.getmtime(fname_out):
                files.append(fname)
    if len(files)==0: print("No notebooks were modified.")
    else:
        if sys.platform == "win32": n_workers = 0
        passed = parallel(nb2md, files, n_workers=n_workers, exp=exp, pause=pause)
        if not all(passed):
            msg = "Conversion failed on the following:\n"
            print(msg + '\n'.join([f.name for p,f in zip(passed,files) if not p]))

You can use `parallel_nb2md` to recursively convert a directory of notebooks to markdown files.

In [ ]:
parallel_nb2md('../tests', recursive=False, n_workers=0)

converting: ../tests/00_some.thing.ipynb
converting: ../tests/minimal.ipynb
converting: ../tests/01_everything.ipynb


In [ ]:
for f in _test_nbs:
    assert f.with_suffix('.md').exists(), f'{str(f)} does not exist.'

The modified times of notebooks are introspected such notebooks that haven't changed after markdown files have been created will not be converted:

In [ ]:
parallel_nb2md('test_files/', exp=get_mdx_exporter(), recursive=True)

No notebooks were modified.


However, you can set `force_all` = `True` to force notebooks to convert:

In [ ]:
parallel_nb2md('test_files/', exp=get_mdx_exporter(), recursive=True, force_all=True)

converting: test_files/run_flow_showstep.ipynbconverting: test_files/hello_world.ipynb

converting: test_files/run_flow.ipynb
converting: test_files/example_input.ipynb
converting: test_files/writefile.ipynb
converting: test_files/non_executed.ipynb
converting: test_files/visibility.ipynb
converting: test_files/doc.ipynb
converting: test_files/frontmatter.ipynb
converting: test_files/exec.ipynb


In [ ]:
#hide
for f in _test_nbs: f.with_suffix('.md').unlink(missing_ok=True)

In [ ]:
#export
@call_parse
def nbdoc_build(
    srcdir:str=None,  # A directory of notebooks to convert to docs recursively, can also be a filename.
    force_all:bool_arg=False, # Rebuild even notebooks that havent changed
    n_workers:int=None,  # Number of workers to use
    pause:float=0.5  # Pause time (in secs) between notebooks to avoid race conditions
):
    "Build the documentation by converting notebooks in `srcdir` to markdown"
    parallel_nb2md(basedir=srcdir, 
                   exp=get_mdx_exporter(), 
                   recursive=True, 
                   force_all=force_all, 
                   n_workers=n_workers, 
                   pause=pause)